In [172]:
import pandas as pd
import numpy as np
import pyodbc
import sklearn as sk
import tensorflow as tf
import json

In [173]:
with open(r"sql_config.json") as f:
    sql_config = json.load(f)

In [174]:
global con, cur
con = pyodbc.connect(
    "Driver={SQL Server Native Client 11.0};"
    f"Server={sql_config['ip']},{sql_config['port']};"
    f"Database={sql_config['database']};"
    f"UID={sql_config['uid']};"
    f"PWD={sql_config['password']}"
)
cur = con.cursor()

In [175]:
df = pd.DataFrame()
sql = "SELECT * FROM FatTrimmerData;"
for chunk in pd.read_sql(sql, con, chunksize=10000):
    df = pd.concat([df, chunk])

c:\Users\qsiba\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [332]:
df = df[df.DeltaTime != -1]
df.drop_duplicates(inplace=True)
df["NewItems"] = df["NewItems"].astype(int)

In [333]:
x = df.drop(columns=["NewItems"])
print(df.shape)
print(x.shape)
y = df["NewItems"]
print(y.head())

(171040, 3)
(171040, 2)
4     0
5     0
8     0
9     0
11    0
Name: NewItems, dtype: int32


In [334]:
df.head()
#dataset = pd.read_csv("E:\cancer.csv")
#x = dataset.drop(columns=["diagnosis(1=m, 0=b)"])
#y = dataset["diagnosis(1=m, 0=b)"]
#print(x.shape)
#print(y.shape)
#print(dataset.shape)
#x.head


,NewItems,PageNumber,DeltaTime
4,0,274,4161000
5,0,1054,3501000
8,0,1326,7058000
9,0,114,4357000
11,0,660,2173000


In [335]:
print(f"num physical devices: {len(tf.config.list_physical_devices('GPU'))}")

num physical devices: 0


In [341]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
test = x_train.shape
print(test[1])

2


In [342]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense

model = Sequential([
    Dense(units=256, input_shape=(x_train.shape[1],), activation='relu'),
    Dense(units=256, activation='relu'),
    Dense(units=1, activation='softmax')
])

In [343]:
model.summary()

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_126 (Dense)           (None, 256)               768       
                                                                 
 dense_127 (Dense)           (None, 256)               65792     
                                                                 
 dense_128 (Dense)           (None, 1)                 257       
                                                                 
Total params: 66,817
Trainable params: 66,817
Non-trainable params: 0
_________________________________________________________________


In [344]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# POINT OF NO RETURN

In [345]:

print(x_train.shape)
print(y_train.shape)
print(x_train.size)
print(y_train.size)
model.fit(x_train, y_train, epochs=1000)
model.evaluate(x_test, y_test)

(136832, 2)
(136832,)
273664
136832
Epoch 1/1000
2520/4276 [================>.............] - ETA: 1s - loss: 4872.2412 - accuracy: 0.0175

KeyboardInterrupt: 